# From quantum to classical modeling of radiation reaction: A focus on stochasticity effects
paper: https://journals.aps.org/pre/abstract/10.1103/PhysRevE.97.043209

In this notebook we apply the Fokker Planck pusher to reproduce figure 10 (energy loss of electrons in a constant magnetic field) from the paper by Niel et al.

The change in energy per timestep is:
$$\mathrm{d} \gamma = - S(\chi) \mathrm{d}t + \sqrt{R(\chi,\gamma)} \mathrm{d}W$$
with $\chi = 10^{-3} \gamma[1800] B[2.5kT] $ and $S,R$ functions related to the rate of photon emission.

The Wiener process $\mathrm{d}W$ is simulated by drawing a normally distributed random number $\mathcal{N}(0,1)$ and multiplying by $\sqrt{dt}$.

In [9]:
# import auxiliary functions
import pushers
from pushers import *

# choose name of file where to save
filename = 'chi0_-1_FP' #

In [10]:
# sampling
Nsmpl = int(5e4); #[] number of particles
print("Number of particles =", Nsmpl)

# initial conditions: Niel2018 page 17/27
gmdist = 1800 + 90*rng.standard_normal(Nsmpl)
gmdist[gmdist<1]=1 # make sure no particle has a physically inconsistent gamma
gmdist_dump1 = np.copy(gmdist) #[] dump1
gmdist_dump2 = np.copy(gmdist) * 0 #[] dump2
gmdist_dump3 = np.copy(gmdist) * 0 #[] dump3

# reference chi0
chi0 = 1e-1; #[] {1e-3,1e-2,1e-1,1e0}
print("chi0 =",chi0)

# reference omega_p
omega_p = elementary_charge/electron_mass/1800 * 2.5e3 * (chi0/(1e-3))
print("omega_p =",omega_p)

# dimensionless but physically relevant constant
Kalpha = fine_structure * electron_mass * speed_of_light**2 / (hbar * omega_p)
print("Kalpha = alpha^2/tau_e/omega_p =",Kalpha)

# simulation parameters
tmax = 5 #[1/omega_p] simulation duration {20,20,5,5}
tdim = 1000 #[]
dt = tmax/tdim #[1/omega_p]
print("number of timesteps =", tdim)
print("dt =", dt)

Number of particles = 50000
chi0 = 0.1
omega_p = 24428055705168.934
Kalpha = alpha^2/tau_e/omega_p = 231915.97672953582
number of timesteps = 1000
dt = 0.005


In [11]:
# interpolation
gmlst = np.linspace(1,2*np.max(gmdist),200); #[]
S_lst = np.array([S39(gm/1800*chi0,Kalpha) for gm in gmlst])
S_intrp = interpolate.interp1d(gmlst, S_lst)
R_lst = np.array([R40(gm/1800*chi0,gm,Kalpha) for gm in gmlst])
R_intrp = interpolate.interp1d(gmlst, R_lst)

In [ ]:
# for each time step
for t in tqdm(range(tdim)):
    
    # for each particle
    for i in range(Nsmpl):
        
        # interpolate S and R
        S = S_intrp(gmdist[i]);
        R = R_intrp(gmdist[i]);
        
        # Gaussian random number
        dW = sqrt(dt) * np.random.randn()
        
        # Niel2018 eq 42
        dgamma = -S * dt + sqrt(R) * dW
        
        # update gamma
        gmdist[i] = gmdist[i] + dgamma
        
        # avoid unphysical energies
        if gmdist[i] < 1:
            gmdist[i] = 1
            
    # save distribution
    if t == int(tdim/2):
        gmdist_dump2 = np.copy(gmdist)
            
# save distribution
gmdist_dump3 = np.copy(gmdist)

In [ ]:
# get histograms
nbins = 50

# dump 1
gmdist_y,gmdist_x = np.histogram(gmdist_dump1,np.linspace(1,1800+4*90,nbins))
gmdist1_y, gmdist1_x = gmdist_y, np.array(arraycenter(gmdist_x))

# dump 2
gmdist_y,gmdist_x = np.histogram(gmdist_dump2,np.linspace(1,1800+4*90,nbins))
gmdist2_y, gmdist2_x = gmdist_y, np.array(arraycenter(gmdist_x))

# dump 3
gmdist_y,gmdist_x = np.histogram(gmdist_dump3,np.linspace(1,1800+4*90,nbins))
gmdist3_y, gmdist3_x = gmdist_y, np.array(arraycenter(gmdist_x))

In [ ]:
# plot
plt.plot(gmdist1_x/1800,gmdist1_y/np.max(gmdist1_y),'-r')
plt.plot(gmdist2_x/1800,gmdist2_y/np.max(gmdist1_y),'-r')
plt.plot(gmdist3_x/1800,gmdist3_y/np.max(gmdist1_y),'-r')
plt.xlim([0,1.25])
plt.ylim(np.array([1e-2,1e1]) )
plt.yscale('log')
plt.show()

In [ ]:
# save results to pickle
outfile = open(filename,'wb')
pickle.dump([chi0, tmax, tdim, Nsmpl, gmdist_dump1, gmdist_dump2, gmdist_dump3],outfile)
outfile.close()